In [7]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv

In [8]:
from langchain_community.chat_models import ChatOpenAI

In [9]:
load_dotenv()
Key = os.getenv("API_KEY")
BASE_URL = os.getenv("BASE_URL")

In [10]:
llm = ChatOpenAI(api_key = Key, model_name = "openai/gpt-3.5-turbo-0613", base_url = BASE_URL, temperature = 0.5)

In [11]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000021A334BAA90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000021A334BB650>, model_name='openai/gpt-3.5-turbo-0613', temperature=0.5, model_kwargs={}, openai_api_key='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', openai_api_base='https://openrouter.ai/api/v1', openai_proxy='')

In [ ]:
from openai import OpenAI
import langchain
# from langchain.llms import OpenAI
from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [13]:
RESPONSE_JSON = {
    "1.":{
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
    "2":{
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
    "3":{
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "4":{
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "5":{
        "mcq": "multiple choice question",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    }
}

### MCQ Generator template

In [14]:
TEMPLATE = """
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice question for {subject} student in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforiming the text as well
Make sure to format your response like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [15]:
from langchain.prompts import PromptTemplate

quiz_generation_prompt = PromptTemplate(
    input_variable = ["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE
)

In [16]:
quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key="quiz", verbose = True)

C:\Users\HP\AppData\Local\Temp\ipykernel_16088\717102180.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key="quiz", verbose = True)


### Correct answer Template

In [17]:
Template2 = """
You are an expert english grammarian and writer. Given a multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for the complexity.
if the quiz is not at per with the cognitive and analytical abilities of the students.\
update the quiz question which need to be changed and change the tone such that it perfectly fit the student ability
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [18]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [19]:
review_chain = LLMChain(llm = llm, prompt = quiz_evaluation_prompt, output_key = "review", verbose=True)

In [20]:
generate_evaluate_chain = SequentialChain(chains = [quiz_chain, review_chain],
                                          input_variables = ["text", "number", "subject", "tone", "response_json"],
                                          output_variables = ["quiz", "review"],
                                          verbose = True)

### Read data fron data.txt

In [21]:
file_path = "C:/Users/HP/Desktop/mcq_generator/data.txt"

In [22]:
file_path

'C:/Users/HP/Desktop/mcq_generator/data.txt'

In [23]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [24]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contri

In [25]:
# Convert the dict file to json-formatted string
json.dumps(RESPONSE_JSON)

'{"1.": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "5": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [26]:
NUMBER = 5,
SUBJECT = "data science"
TONE = "simple"

In [27]:
# https://python.langchain.com/docs/modules/model_io/llms/token_usuage_tracking

# how to setup token usuage Tracking in Langchain
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons

C:\Users\HP\AppData\Local\Temp\ipykernel_16088\2931217859.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = generate_evaluate_chain(



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Text: The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to c

In [28]:
print(f"Total Token: {cb.total_tokens}")
print(f"prompt token: {cb.prompt_tokens}")
print(f"Completion Token: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Token: 2768
prompt token: 1996
Completion Token: 772
Total Cost: 0.0


In [29]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nThe earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive s

In [30]:
quiz = response.get("quiz")

In [31]:
json.loads(quiz)

{'1.': {'mcq': 'Who coined the term machine learning?',
  'options': {'a': 'Donald Hebb',
   'b': 'Raytheon Company',
   'c': 'Arthur Samuel',
   'd': 'Tom M. Mitchell'},
  'correct': 'c'},
 '2': {'mcq': 'What was the earliest machine learning program about?',
  'options': {'a': 'Calculating winning chance in checkers',
   'b': 'Analyzing sonar signals',
   'c': 'Recognizing speech patterns',
   'd': 'Studying human cognitive processes'},
  'correct': 'a'},
 '3': {'mcq': 'Which researchers proposed early mathematical models of neural networks?',
  'options': {'a': 'Walter Pitts and Warren McCulloch',
   'b': 'Raytheon Company',
   'c': 'Donald Hebb',
   'd': 'Tom M. Mitchell'},
  'correct': 'a'},
 '4': {'mcq': 'Who provided a formal definition of the algorithms studied in machine learning field?',
  'options': {'a': 'Donald Hebb',
   'b': 'Raytheon Company',
   'c': 'Tom M. Mitchell',
   'd': 'Arthur Samuel'},
  'correct': 'c'},
 '5': {'mcq': 'What are the two objectives of modern-day 

### Creating Dataframe out of the quiz

In [32]:
quiz_table_data = []
quiz = json.loads(quiz)

for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ],
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ":mcq, "Choices": options, "Correct": correct})


In [33]:
quiz_table_data

[{'MCQ': 'Who coined the term machine learning?',
  'Choices': 'a: Donald Hebb | b: Raytheon Company | c: Arthur Samuel | d: Tom M. Mitchell',
  'Correct': 'c'},
 {'MCQ': 'What was the earliest machine learning program about?',
  'Choices': 'a: Calculating winning chance in checkers | b: Analyzing sonar signals | c: Recognizing speech patterns | d: Studying human cognitive processes',
  'Correct': 'a'},
 {'MCQ': 'Which researchers proposed early mathematical models of neural networks?',
  'Choices': 'a: Walter Pitts and Warren McCulloch | b: Raytheon Company | c: Donald Hebb | d: Tom M. Mitchell',
  'Correct': 'a'},
 {'MCQ': 'Who provided a formal definition of the algorithms studied in machine learning field?',
  'Choices': 'a: Donald Hebb | b: Raytheon Company | c: Tom M. Mitchell | d: Arthur Samuel',
  'Correct': 'c'},
 {'MCQ': 'What are the two objectives of modern-day machine learning?',
  'Choices': 'a: Data classification and making predictions | b: Analyzing human cognitive pro

In [34]:
import pandas as pd
quiz = pd.DataFrame(quiz_table_data)

In [35]:
quiz.to_csv("machinelearning.csv", index = False)

In [36]:
from datetime import datetime
LOG_FILE = f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

In [37]:
LOG_FILE

'07_25_2025_13_29_29.log'